## Simple Data Augmentation Techniques Using Albumentation

Even though the dataset size seems large [119 GB] it is mostly because of DICOM images, if those images are stored as PNG, the dataset size would be 3 to 4 GB as it only contains about 6K images. 
Thus, data augmentation will play as a game changer for training different model architectures in this competition.

* As part of the article on [data augmentation techniques](https://www.kaggle.com/c/siim-covid19-detection/discussion/242760), this notebook shows different simple augmentation applied to different images.

* Each section can be accessed from the right side by it's specific name.

* Each Augmentation is also marked with it's Albumentation Definition so that one can change it's default parameters and play around with the data.

* After applying the augmentation, one can save the augmented images [Code Present in Augmentation Utility Section] or can integrate these augmentation techniques on the fly with their models.

Please ask any questions in the comments below.

Thank You!!


## INIT

In [ ]:
!conda install -c conda-forge gdcm -y

In [ ]:
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image
import cv2
import random
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import json
import pydicom
import glob
from tqdm.notebook import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage import exposure
import warnings
from fastai.vision.all import *
from fastai.medical.imaging import *
warnings.filterwarnings('ignore')

In [ ]:
from albumentations import (
    BboxParams,
    HorizontalFlip,
    VerticalFlip,
    Resize,
    CenterCrop,
    RandomCrop,
    Crop,
    Compose,
    Rotate,
    CropNonEmptyMaskIfExists,
    RandomSizedBBoxSafeCrop,
    Blur,
    ChannelDropout,
    ChannelShuffle,
    FancyPCA,
    GaussNoise,
    GaussianBlur,
    GlassBlur,
    HueSaturationValue,
    ImageCompression,
    InvertImg,
    MedianBlur,
    MotionBlur,
    MultiplicativeNoise,
    Normalize,
    Posterize,
    RGBShift,
    RandomBrightnessContrast,
    RandomFog,
    RandomGamma,
    RandomRain,
    RandomSnow,
    RandomSunFlare,
    Solarize
)

In [ ]:
train_folder = '../input/siim-covid19-detection/train'
test_folder = '../input/siim-covid19-detection/test'
train_image_level = '../input/siim-covid19-detection/train_image_level.csv'
train_study_level = '../input/siim-covid19-detection/train_study_level.csv'
sample_submission = '../input/siim-covid19-detection/sample_submission.csv'

In [ ]:
train_data = pd.read_csv(train_image_level)

In [ ]:
train_data.describe()

## [Reference for Helper Functions](https://www.kaggle.com/tanlikesmath/siim-covid-19-detection-a-simple-eda#A-look-at-the-images)

In [ ]:
# DICOM to NP.array conversion
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    '''
    function to convert DICOM images into np.array.
    :param path: path of the DICOM image.
    :param VOI LUT: used to transform DICOM data to more simpler data.
    :param fix_monochrome: used to fix the inversion of X-Ray images.
    :return: the Grayscale image converted to np.array format from DICOM format. 
    '''
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    return data

# Image Path Extraction
def image_path(row):
    '''
    function to retrieve DICOM image path from the given directory structure.
    :param row: ith row of a dataframe which would be parsed to get the image's path.
    :return: image path w.r.t. the given directory.
    '''
    study_path = train_folder + '/' + row.StudyInstanceUID
    for i in get_dicom_files(study_path):
        if row.id.split('_')[0] == i.stem: return i
        
# Image Visualization         
def plot_imgs(annotations, cols=4, size = 7, thickness = 13, img_resize = (500, 500)):
    '''
    function to display image using matplot-lib.
    :param annotations: List of dictionaries containg the image, bboxes and category_ids to plot.
    :param cols: number of columns to use for displaying the images.
    :param size: to determine the size of the total plot w.r.t columns and rows.
    :param thickness: the thickness of the rectangle border [bboxes] line in px. 
    :param img_resize: the new values for changing the size of the image
    '''
    rows = len(annotations)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i, annotation in enumerate(annotations):
        img = annotation['image']
        bboxes = annotation['bboxes']
        for box in bboxes:    
            '''
            xmin = xmin
            ymin = ymin
            xmax = xmin + w
            ymax = ymin + h
            '''
            xmin = int(box[0])
            ymin = int(box[1])
            xmax = int((box[0]+box[2]))
            ymax = int((box[1]+box[3]))
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), [255, 0, 0], thickness)
        img = cv2.resize(img, img_resize)
        ax = fig.add_subplot(rows, cols, i+1)
        plt.imshow(img, cmap = 'gray')
        ax.title.set_text('Image_'+str(i))
    plt.show()
    
# Data Cleaning
def clean_data(data):
    '''
    function to format the dataframe in order to make the images and bounding boxes more accessible.
    :param data: Dataframe containing the original image ids and bounding boxes.
    :return: Cleaned Up dataframe of the dataset containing.
    '''
    # Drop images with no BBoxes
    data = data.dropna()
    #Forming all the image paths for easy access.
    data['image_path'] = data.apply(image_path, axis=1)
    #Formatting bboxes = [[x, y, w, h], [x, y, w, h]] for data augmentation.
    data['bboxes'] = data.boxes.apply(lambda x: list(list(x.values()) for x in eval(str(x))))
    
    return data

# Annotation Creation
def get_annotations(data , no_images=4):
    '''
    function to retrieve annotations i.e. list of dictionaries containing the image, bboxes. 
    and category_id for randomly selected 'no_images' images.
    :param data: Dataframe from which the image path and bboxes are selected.
    :param no_images: Number of images to be converted into annotations.
    :return: Annotations as List of Dictionaries containing image as an np.array, list of bboxes, and category_id.
    '''
    # These Images will be used for augmentation display. 
    # We randomly select n images and then convert them into np.array format.
    # We then create an annotation list of dictionaries, as the Albumentation takes values as annotations.
    image_paths = random.choices(data['image_path'].values, k=no_images)
    imgs = [dicom2array(path) for path in image_paths]
    annotations = []
    for im in range(len(image_paths)):
        image_path = image_paths[im]
        # Get all the bound boxes as values to 'bboxes'.
        bboxes_imgs = data.loc[data['image_path'] == image_path].bboxes.values.tolist()[0]
        # Get all the images as values to 'image' in RGB 3 channels.
        image = cv2.cvtColor(imgs[im], cv2.COLOR_GRAY2RGB)
        annotations.append({'image': image, 'bboxes': bboxes_imgs, 'category_id': [1]*len(bboxes_imgs)})
    return annotations

## Albumentation Utility Functions

In [ ]:
# def saveImage(path,annotations):
    # Takes path and annotations and saves the images at that path.
    # img = annotations['image'].copy()
    # cv2.imwrite(path, img)

def get_albumentation_aug(aug, min_area=0., min_visibility=0.):
    '''
    function to get the transform object to apply albumentation transformation and handle all 
    transformations regarding bounding boxes.
    :param aug: type of transformation object to create.
    :param min_area: 
    :return: transform object 
    '''
    return Compose(aug, bbox_params=BboxParams(format='coco', min_area=min_area, min_visibility=min_visibility, label_fields=['category_id']))

def augmentation(annotations, filter = 'VerticalFlip'):
    '''
    function to perform different augmentation on the images and then display them.
    :param annotations: Image annotation containing image as np.array, bboxes and category id.
    :param count: Number of Images to be augmentated 
    :param filter: Type of Filter to apply on images (RGB, Contrast, VerticalFlip, etc...)
    '''
    # Change default parameters for filters to see different effects on the data.
    
    # Save Augmented BBoxes and paths of data
    # augmented_BBoxes = []
    # path = []
    augmented_annotations = []
    for i, annotation in enumerate(annotations):
        # Use this for further saving images
        # filter_title = 'image_'+str(i)+'_'+filter
        if filter == 'VerticalFlip':
            aug = get_albumentation_aug([VerticalFlip(p=1)])
        elif filter == 'HorizontalFlip':
            aug = get_albumentation_aug([HorizontalFlip(p=1)])
        elif filter == 'Blur':
            aug = get_albumentation_aug([Blur(p=1)])
        elif filter == 'ChannelDropout':
            aug = get_albumentation_aug([ChannelDropout(p=1)])
        elif filter == 'GaussianBlur':
            aug = get_albumentation_aug([GaussianBlur(p=1)])
        elif filter == 'GlassBlur':
            aug = get_albumentation_aug([GlassBlur(p=1)])
        elif filter == 'InvertImg':
            aug = get_albumentation_aug([InvertImg(p=1)])
        elif filter == 'RandomFog':
            aug = get_albumentation_aug([RandomFog(p=1)])
        elif filter == 'RandomGamma':
            aug = get_albumentation_aug([RandomGamma(p=1)])
        elif filter == 'RandomSunFlare':
            aug = get_albumentation_aug([RandomSunFlare(p=1)])
        elif filter == 'RGBShift':
            aug = get_albumentation_aug([RGBShift(p=1)])
        elif filter == 'RandomSizedBBoxSafeCrop':    
            aug = get_albumentation_aug([RandomSizedBBoxSafeCrop(height=350,width=350,p=1)])
        elif filter == 'GaussNoise':
            aug = get_albumentation_aug([GaussNoise(p=1)])
        elif filter == 'MultiplicativeNoise':
            aug = get_albumentation_aug([MultiplicativeNoise(p=1)])
        elif filter == 'RandomBrightnessContrast':
            aug = get_albumentation_aug([RandomBrightnessContrast(p=1)])
        augmented_annotation = aug(**annotation)
        # saveImage("./"+ filter_title + ".png", augmented_annotation)
        # augmented_BBoxes.append(augmented_annotation['bboxes'])
        # path.append("./"+ filter_title + ".png")
        augmented_annotations.append(augmented_annotation)
    # pd.DataFrame({"bboxes": augmented_BBoxes, "path": path}).to_csv("./Augmented_Data_"+filter+".csv")
    plot_imgs(augmented_annotations)

## Base Images

In [ ]:
train_data = clean_data(train_data)
annotations = get_annotations(train_data, 8)

In [ ]:
# Visualizing Base Images that will be used for augmentation
plot_imgs(annotations)

## [Vertical Flip](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.VerticalFlip)

In [ ]:
augmentation(annotations, filter = 'VerticalFlip')

## [Horizontal Flip](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.HorizontalFlip)

In [ ]:
augmentation(annotations, filter = 'HorizontalFlip')

## [Blur](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.Blur)

In [ ]:
augmentation(annotations, filter = 'Blur')

## [GlassBlur](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.GlassBlur)

In [ ]:
augmentation(annotations, filter = 'GlassBlur')

## [GaussianBlur](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.GaussianBlur)

In [ ]:
augmentation(annotations, filter = 'GaussianBlur')

## [InvertImg](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.InvertImg)

In [ ]:
augmentation(annotations, filter = 'InvertImg')

## [RandomFog](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.RandomFog)

In [ ]:
augmentation(annotations, filter = 'RandomFog')

## [RandomGamma](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.RandomGamma)

In [ ]:
augmentation(annotations, filter = 'RandomGamma')

## [ChannelDropout](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.ChannelDropout)

In [ ]:
augmentation(annotations, filter = 'ChannelDropout')

## [RGBShift](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.RGBShift)

In [ ]:
augmentation(annotations, filter = 'RGBShift')

## [RandomSizedBBoxSafeCrop](https://albumentations.ai/docs/api_reference/augmentations/crops/transforms/#albumentations.augmentations.crops.transforms.RandomSizedBBoxSafeCrop)

In [ ]:
augmentation(annotations, filter = 'RandomSizedBBoxSafeCrop')

## [RandomSunFlare](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.RandomSunFlare)

In [ ]:
augmentation(annotations, filter = 'RandomSunFlare')

## [GaussianNoise](https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.GaussNoise)

In [ ]:
augmentation(annotations, filter = 'GaussNoise')

## End Note

* There are several other [augmentation techniques](https://albumentations.ai/docs/) as well as these techniques can be combined with each other and passed to a model for training. 

* Furthermore, one can also use [AutoAlbument](https://albumentations.ai/docs/autoalbument/) which learns image augmentation policies from image itself using Faster AutoAugment Algorithm. This allows us to automatically select the best augmentation techniques for the given task.

Please ask any questions in the comments below.

Thank You!!